In [1]:
from flask import render_template
import torch
from blazeface.blazeface import BlazeFace
# Load models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
blazeface = BlazeFace().to(device)
blazeface.load_weights("blazeface.pth")
blazeface.load_anchors("anchors.npy")
blazeface.min_score_thresh = 0.7
blazeface.min_suppression_threshold = 0.3
blazeface.eval()

facenet = torch.jit.load("models/mobilefacenet_scripted.pt", map_location=device)

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# BlazeFace (PyTorch nn.Module bình thường)
print("BlazeFace trainable params:", count_parameters(blazeface))

# MobileFaceNet (đang ở dạng TorchScript)
# TorchScript không có .parameters() trực tiếp, cần chuyển về nn.Module nếu có thể
try:
    print("MobileFaceNet trainable params:", count_parameters(facenet))
except AttributeError:
    # Nếu không truy cập được parameters(), cần load model gốc nn.Module
    print("MobileFaceNet TorchScript không hỗ trợ .parameters(). Cần load nn.Module để đếm tham số.")

BlazeFace trainable params: 101390
MobileFaceNet trainable params: 999936


In [20]:
!pip uninstall torch torchvision -y
!pip install torch==2.2.0+cpu torchvision==0.17.0+cpu --index-url https://download.pytorch.org/whl/cpu

Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchvision 0.17.2
Uninstalling torchvision-0.17.2:
  Successfully uninstalled torchvision-0.17.2
Looking in indexes: https://download.pytorch.org/whl/cpu
     ---------------------------------------- 0.0/200.8 MB ? eta -:--:--
      -------------------------------------- 3.4/200.8 MB 18.3 MB/s eta 0:00:11
     - ------------------------------------- 9.2/200.8 MB 22.9 MB/s eta 0:00:09
     -- ----------------------------------- 15.7/200.8 MB 26.1 MB/s eta 0:00:08
     ---- --------------------------------- 24.1/200.8 MB 29.4 MB/s eta 0:00:07
     ------ ------------------------------- 32.2/200.8 MB 31.5 MB/s eta 0:00:06
     ------- ------------------------------ 39.6/200.8 MB 32.3 MB/s eta 0:00:05
     -------- ----------------------------- 46.1/200.8 MB 31.9 MB/s eta 0:00:05
     ---------- --------------------------- 53.2/200.8 MB 32.0 MB/s eta 0:00:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.8.0+cpu requires torch==2.8.0, but you have torch 2.2.0+cpu which is incompatible.


In [ ]:
!pip install facenet-pytorch

In [ ]:
!pip install faiss-cpu==1.7.4

In [21]:
import torch
import cv2
import os
import numpy as np
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
from facenet_pytorch import InceptionResnetV1
from blazeface import BlazeFace
import faiss
import pickle
import shutil

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [5]:
import pickle
index_path = r'D:\PROJECT\ATTENDANCE SYSTEM\face_index.faiss'
id_mapping_path = r'D:\PROJECT\ATTENDANCE SYSTEM\face_ids.pkl'

with open(id_mapping_path, 'rb') as f:
    face_ids = pickle.load(f)
print(type(face_ids))
print(face_ids)

<class 'list'>
['Leonardo DiCaprio', 'Will Smith', 'Angelina Jolie', 'Brad Pitt', 'Denzel Washington', 'Hugh Jackman', 'Jennifer Lawrence', 'Johnny Depp', 'Kate Winslet', 'Megan Fox', 'Natalie Portman', 'Nicole Kidman', 'Robert Downey Jr', 'Sandra Bullock', 'Scarlett Johansson', 'Tom Cruise', 'Tom Hanks', 'Vy Trinh', 'Trang Vu', 'Tan Nguyen', 'Lay Duong', 'Ky Lao', 'Ha Chu', 'Bich Phuong', 'Chi Pu', 'Doan Quoc Dam', 'Dung Hoang', 'Ha Ho', 'Kha Ngan', 'Minzy Hoa', 'Nhi', 'Phuc Duc', 'Taylor Swift', 'Thinh Noo', 'Thuy Chi', 'Tuan Ha', 'Tung Son', 'Van Mai', 'Dao Duong', 'Bao Ngoc', 'Chau Bui', 'Hong Nhung', 'Manh Truong', 'My Linh', 'Phuong Hang', 'Trang Phap', 'Truong My Lan', 'Uyen Linh', 'Cat Tuong', 'Tung']


In [17]:
import torch
print("Torch:", torch.__version__)

Torch: 2.8.0+cpu


### CREATE FAISS

In [ ]:
def create_faiss_from_images(input_folder_path, index_path, id_mapping_path, blazeface_path, anchors_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load BlazeFace
    front_net = BlazeFace().to(device)
    front_net.load_weights(blazeface_path)
    front_net.load_anchors(anchors_path)
    front_net.min_score_thresh = 0.7
    front_net.min_suppression_threshold = 0.3
    front_net.eval()

    # Load FaceNet
    facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

    face_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    face_vectors = []
    face_ids = []

    for filename in os.listdir(input_folder_path):
        if not filename.lower().endswith((".jpg", ".png", ".jpeg")):
            continue

        img_path = os.path.join(input_folder_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Lỗi đọc ảnh: {filename}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        h, w, _ = img.shape

        detections = front_net.predict_on_image(img)
        if len(detections) == 0:
            print(f"Không phát hiện khuôn mặt: {filename}")
            continue

        for det in detections:
            ymin, xmin, ymax, xmax = det[:4]
            x1 = int(xmin * w)
            y1 = int(ymin * h)
            x2 = int(xmax * w)
            y2 = int(ymax * h)

            face_crop = img[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue

            face_tensor = face_transform(face_crop).unsqueeze(0).to(device)
            with torch.no_grad():
                embedding = facenet(face_tensor)
                embedding = F.normalize(embedding)

            vector = embedding.squeeze(0).cpu().numpy().astype('float32')
            face_vectors.append(vector)

            student_id = os.path.splitext(filename)[0]
            face_ids.append(student_id)

            print(f"Đã thêm vector cho: {student_id}")

    # Convert sang numpy
    if len(face_vectors) == 0:
        print("❌ Không có vector nào để build index.")
        return

    face_vectors_np = np.array(face_vectors, dtype='float32')
    d = face_vectors_np.shape[1]
    n_data = face_vectors_np.shape[0]

    print(f"\n📦 Tổng số vector: {n_data}")

    # Tùy vào số lượng vector mà chọn index phù hợp
    if n_data < 10:
        print("⚠️ Dữ liệu ít, dùng IndexFlatL2 (RAM cao).")
        index = faiss.IndexFlatL2(d)
        index.add(face_vectors_np)
    else:
        quantizer = faiss.IndexFlatL2(d)
        nlist = min(100, n_data // 2)
        if nlist < 1:
            nlist = 1
        print(f"⚙️ Dùng IndexIVFFlat với {nlist} cụm...")
        index = faiss.IndexIVFFlat(quantizer, d, nlist)
        print("⏳ Train index...")
        index.train(face_vectors_np)
        index.add(face_vectors_np)

    # Lưu index và ID mapping
    faiss.write_index(index, index_path)
    with open(id_mapping_path, 'wb') as f:
        pickle.dump(face_ids, f)

    print(f"\n✅ Đã tạo FAISS index với {n_data} vector.")

### RECOGNIZE IMAGE AND EVALUATE

In [ ]:
class FaceRecognizer:
    def __init__(self, blazeface_path, anchors_path, index_path, id_mapping_path, threshold=0.95, device='cuda'):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.threshold = threshold

        # Load BlazeFace
        self.detector = BlazeFace().to(self.device)
        self.detector.load_weights(blazeface_path)
        self.detector.load_anchors(anchors_path)
        self.detector.min_score_thresh = 0.7
        self.detector.min_suppression_threshold = 0.3
        self.detector.eval()

        # Load FaceNet
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval().to(self.device)

        # Load FAISS index & ID mapping
        self.index = faiss.read_index(index_path)
        with open(id_mapping_path, 'rb') as f:
            self.face_ids = pickle.load(f)

        # Preprocess
        self.face_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((160, 160)),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])

        print("📂 FAISS index loaded:")
        print(" - Số vector:", self.index.ntotal)
        print(" - Loại index:", type(self.index))
        print(" - Chiều vector:", self.index.d)


    def recognize(self, img_path, show=False):
        img_orig = cv2.imread(img_path)
        if img_orig is None:
            print("❌ Không đọc được ảnh:", img_path)
            return "Invalid", None

        img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
        img_small = cv2.resize(img_rgb, (128, 128))
        h, w = img_orig.shape[:2]

        # Detect face
        detections = self.detector.predict_on_image(img_small)
        if len(detections) == 0:
            print("❌ Không phát hiện khuôn mặt.")
            return "NoFace", None

        for det in detections:
            ymin, xmin, ymax, xmax = det[:4]
            x1, y1, x2, y2 = int(xmin * w), int(ymin * h), int(xmax * w), int(ymax * h)
            face_crop = img_rgb[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue

            # Embedding
            face_tensor = self.face_transform(face_crop).unsqueeze(0).to(self.device)
            with torch.no_grad():
                emb = self.facenet(face_tensor)
                emb = F.normalize(emb)

            query = emb.squeeze(0).cpu().numpy().astype('float32').reshape(1, -1)
            distances, indices = self.index.search(query, k=1)
            dist, idx = distances[0][0], indices[0][0]

            recognized_person = self.face_ids[idx] if dist < self.threshold else "Unknown"

            if show:
                cv2.rectangle(img_orig, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = f"{recognized_person} ({dist:.2f})"
                cv2.putText(img_orig, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0), 2, cv2.LINE_AA)
                plt.figure(figsize=(8, 8))
                plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.title("Result")
                plt.show()

            return recognized_person, dist

        return "NoFace", None

    def recognize_frame(self, frame):
        img_rgb = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB)
        img_small = cv2.resize(img_rgb, (128, 128))
        h, w = frame.shape[:2]

        detections = self.detector.predict_on_image(img_small)
        results = []

        for det in detections:
            ymin, xmin, ymax, xmax = det[:4]
            x1, y1, x2, y2 = int(xmin * w), int(ymin * h), int(xmax * w), int(ymax * h)
            face_crop = img_rgb[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue

            face_tensor = self.face_transform(face_crop).unsqueeze(0).to(self.device)
            with torch.no_grad():
                emb = self.facenet(face_tensor)
                emb = F.normalize(emb)

            query = emb.squeeze(0).cpu().numpy().astype('float32').reshape(1, -1)
            distances, indices = self.index.search(query, k=1)
            dist, idx = distances[0][0], indices[0][0]

            recognized_person = self.face_ids[idx] if dist < self.threshold else "Unknown"

            results.append((x1, y1, x2, y2, recognized_person, dist))

        return results

In [6]:
import torch
import gc
import time
import os

def evaluate_recognition(test_folder, recognizer):
    total, correct, unknown, total_time = 0, 0, 0, 0.0
    d_correct, d_wrong = [], []

    test_files = sorted([f for f in os.listdir(test_folder) if f.lower().endswith(('.jpg', '.png'))])

    for filename in test_files:
        true_id = filename.split('_')[0]
        img_path = os.path.join(test_folder, filename)

        start = time.time()
        pred_id, dist = recognizer.recognize(img_path, show=False)
        elapsed = time.time() - start
        total_time += elapsed
        total += 1

        if pred_id == true_id:
            correct += 1
            if dist is not None:
                d_correct.append(dist)
        elif pred_id == "Unknown":
            unknown += 1
            if dist is not None:
                d_wrong.append(dist)
        else:
            if dist is not None:
                d_wrong.append(dist)

        print(f"[{filename}] → GT: {true_id}, Pred: {pred_id}, Dist: {dist}, Time: {elapsed:.3f}s")

        # 🔁 Giải phóng bộ nhớ tạm
        torch.cuda.empty_cache()
        gc.collect()

    acc = correct / total if total else 0
    avg_time = total_time / total if total else 0
    avg_d_c = sum(d_correct)/len(d_correct) if d_correct else 0
    avg_d_w = sum(d_wrong)/len(d_wrong) if d_wrong else 0

    print("\n📊 Evaluation Summary:")
    print(f"📁 Total: {total}, ✅ Correct: {correct}, ❌ Wrong: {total - correct}, 🤷 Unknown: {unknown}")
    print(f"🎯 Accuracy: {acc*100:.2f}%, ⏱ Avg Time: {avg_time:.3f}s")
    print(f"📏 Avg Dist (Correct): {avg_d_c:.4f}, Avg Dist (Wrong): {avg_d_w:.4f}")

### SPLITTING DATASET

In [7]:
def split_dataset(input_root, database_folder, test_folder):
    os.makedirs(database_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    people_folders = [f for f in os.listdir(input_root) if os.path.isdir(os.path.join(input_root, f))]

    for person in people_folders:
        person_path = os.path.join(input_root, person)
        all_images = sorted(os.listdir(person_path))  # đảm bảo thứ tự

        if len(all_images) < 2:
            print(f"⚠️ Bỏ qua {person} (ít hơn 2 ảnh)")
            continue

        # Ảnh đầu tiên → database
        db_img_src = os.path.join(person_path, all_images[0])
        db_img_dst = os.path.join(database_folder, f"{person}.jpg")
        shutil.copy(db_img_src, db_img_dst)

        # 99 ảnh còn lại → test_images
        for i, img_name in enumerate(all_images[1:], start=1):
            src = os.path.join(person_path, img_name)
            dst_name = f"{person}_{i}.jpg"
            dst = os.path.join(test_folder, dst_name)
            shutil.copy(src, dst)

        print(f"✅ Tách {person}: 1 ảnh vào database, {len(all_images)-1} ảnh vào test_images")

    print("\n🎉 Hoàn tất tách dataset!")

In [7]:
data_path = r'C:\Users\HP\Desktop\HK6\CS331\BlazeFace-PyTorch\Celebrity_Faces_Dataset'
db_img = r'C:\Users\HP\Desktop\HK6\CS331\BlazeFace-PyTorch\db_img'
test_img = r'C:\Users\HP\Desktop\HK6\CS331\BlazeFace-PyTorch\test_img'

blazeface_path = r'C:\Users\HP\Desktop\HK6\CS331\BlazeFace-PyTorch\blazeface.pth'
anchors_path = r'C:\Users\HP\Desktop\HK6\CS331\BlazeFace-PyTorch\anchors.npy'

In [9]:
# split_dataset(
#     input_root=data_path,
#     database_folder=db_img,
#     test_folder=test_img
# )

In [8]:
create_faiss_from_images(
    input_folder_path=db_img,
    index_path='face_index.faiss',
    id_mapping_path='face_ids.pkl',
    blazeface_path=blazeface_path,
    anchors_path=anchors_path
)

NameError: name 'transforms' is not defined

In [13]:
recognizer = FaceRecognizer(
    blazeface_path=blazeface_path,
    anchors_path=anchors_path,
    index_path='face_index.faiss',
    id_mapping_path='face_ids.pkl',
    threshold=0.95
)

📂 FAISS index loaded:
 - Số vector: 17
 - Loại index: <class 'faiss.swigfaiss.IndexIVFFlat'>
 - Chiều vector: 512


In [15]:
print("Số vector trong index:", recognizer.index.ntotal)
print("Kiểu FAISS index:", type(recognizer.index))

Số vector trong index: 17
Kiểu FAISS index: <class 'faiss.swigfaiss.IndexIVFFlat'>


In [16]:
evaluate_recognition(test_img, recognizer)

[Angelina Jolie_1.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.3234807848930359, Time: 0.136s
[Angelina Jolie_10.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.42916256189346313, Time: 0.108s
[Angelina Jolie_11.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.2653766870498657, Time: 0.126s
[Angelina Jolie_12.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.26576927304267883, Time: 0.100s
[Angelina Jolie_13.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.5724337100982666, Time: 0.104s
[Angelina Jolie_14.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.4599061608314514, Time: 0.134s
[Angelina Jolie_15.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.3038078844547272, Time: 0.102s
[Angelina Jolie_16.jpg] → GT: Angelina Jolie, Pred: Megan Fox, Dist: 0.6800921559333801, Time: 0.094s
[Angelina Jolie_17.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.5372225046157837, Time: 0.106s
[Angelina Jolie_18.jpg] → GT: Angelina Jo

KeyboardInterrupt: 

In [17]:
print("RAM còn lại:", psutil.virtual_memory().available / 1024**2, "MB")

RAM còn lại: 950.5859375 MB


In [18]:
def run_realtime_camera(recognizer):
    cap = cv2.VideoCapture(0)  # Dùng camera mặc định
    if not cap.isOpened():
        print("❌ Không thể mở camera.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Không đọc được frame từ camera.")
            break

        results = recognizer.recognize_frame(frame)

        for (x1, y1, x2, y2, name, dist) in results:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{name} ({dist:.2f})"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow("Face Recognition Realtime", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Nhấn q để thoát
            break

    cap.release()
    cv2.destroyAllWindows()

In [19]:
recognizer = FaceRecognizer(
    blazeface_path=blazeface_path,
    anchors_path=anchors_path,
    index_path='face_index.faiss',
    id_mapping_path='face_ids.pkl',
    threshold=0.95
)

run_realtime_camera(recognizer)

📂 FAISS index loaded:
 - Số vector: 17
 - Loại index: <class 'faiss.swigfaiss.IndexIVFFlat'>
 - Chiều vector: 512


### COMPUTE WITHOUT FAISS


In [ ]:
def create_database(input_folder_path, output_folder_path):
    # Load mô hình BlazeFace chính diện
    front_net = BlazeFace().to(device)
    front_net.load_weights(blazeface_path)
    front_net.load_anchors(anchors_path)
    front_net.min_score_thresh = 0.7
    front_net.min_suppression_threshold = 0.3
    front_net.eval()

    facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

    # Transform input face image
    face_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    for filename in os.listdir(input_folder_path):
        img_path = os.path.join(input_folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        h, w, _ = img.shape

        detected_face = front_net.predict_on_image(img)

        for i, det in enumerate(detected_face):
            ymin, xmin, ymax, xmax = det[:4]
            x1 = int(xmin * w)
            y1 = int(ymin * h)
            x2 = int(xmax * w)
            y2 = int(ymax * h)

            # Cắt khuôn mặt từ ảnh gốc
            face_crop = img[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue  # Tránh lỗi nếu crop ra vùng rỗng

            # Preprocess và đưa qua FaceNet
            face_tensor = face_transform(face_crop).unsqueeze(0).to(device)
            with torch.no_grad():
                embedding = facenet(face_tensor)  # (1, 512)
                embedding = F.normalize(embedding)  # L2 normalize (tuỳ chọn)
            output_name = f"{os.path.splitext(filename)[0]}_embedding.pt"
            output_path = os.path.join(output_folder_path, output_name)

            torch.save(embedding, output_path)
            # In ra hoặc lưu vector đặc trưng
            print("Đã lưu 1 vector embedding: ", output_name)

In [ ]:
class BaselineRecognizer:
    def __init__(self, blazeface_path, anchors_path, embeddings_folder, threshold=0.95, device='cuda'):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.threshold = threshold

        # Load BlazeFace
        self.detector = BlazeFace().to(self.device)
        self.detector.load_weights(blazeface_path)
        self.detector.load_anchors(anchors_path)
        self.detector.min_score_thresh = 0.7
        self.detector.min_suppression_threshold = 0.3
        self.detector.eval()

        # Load FaceNet
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval().to(self.device)

        # Load embeddings
        self.embeddings = []
        self.ids = []
        for fname in os.listdir(embeddings_folder):
            if fname.endswith(".pt"):
                vector = torch.load(os.path.join(embeddings_folder, fname)).squeeze(0).to(self.device)
                person_id = fname.split("_")[0]
                self.embeddings.append(vector)
                self.ids.append(person_id)

        # Preprocess
        self.face_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((160, 160)),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])

    def recognize(self, img_path, show=False):
        img_orig = cv2.imread(img_path)
        if img_orig is None:
            return "Invalid", None

        img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
        img_small = cv2.resize(img_rgb, (128, 128))
        h, w = img_orig.shape[:2]

        detections = self.detector.predict_on_image(img_small)
        if len(detections) == 0:
            return "NoFace", None

        for det in detections:
            ymin, xmin, ymax, xmax = det[:4]
            x1, y1, x2, y2 = int(xmin * w), int(ymin * h), int(xmax * w), int(ymax * h)
            face_crop = img_rgb[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue

            face_tensor = self.face_transform(face_crop).unsqueeze(0).to(self.device)
            with torch.no_grad():
                emb = self.facenet(face_tensor)
                emb = F.normalize(emb).squeeze(0)

            # So sánh với tất cả vector
            min_dist = float("inf")
            best_match = "Unknown"
            for i, db_vector in enumerate(self.embeddings):
                dist = torch.dist(emb, db_vector).item()
                if dist < min_dist:
                    min_dist = dist
                    best_match = self.ids[i]

            if min_dist < self.threshold:
                return best_match, min_dist
            else:
                return "Unknown", min_dist

        return "NoFace", None

In [ ]:
vector_db = '/content/drive/MyDrive/UIT/CS331/BlazeFace-PyTorch/vector_db'

create_database(db_img, vector_db)

Đã lưu 1 vector embedding:  Angelina Jolie_embedding.pt
Đã lưu 1 vector embedding:  Will Smith_embedding.pt
Đã lưu 1 vector embedding:  Tom Cruise_embedding.pt
Đã lưu 1 vector embedding:  Tom Hanks_embedding.pt
Đã lưu 1 vector embedding:  Johnny Depp_embedding.pt
Đã lưu 1 vector embedding:  Leonardo DiCaprio_embedding.pt
Đã lưu 1 vector embedding:  Natalie Portman_embedding.pt
Đã lưu 1 vector embedding:  Sandra Bullock_embedding.pt
Đã lưu 1 vector embedding:  Kate Winslet_embedding.pt
Đã lưu 1 vector embedding:  Nicole Kidman_embedding.pt
Đã lưu 1 vector embedding:  Robert Downey Jr_embedding.pt
Đã lưu 1 vector embedding:  Jennifer Lawrence_embedding.pt
Đã lưu 1 vector embedding:  Scarlett Johansson_embedding.pt
Đã lưu 1 vector embedding:  Denzel Washington_embedding.pt
Đã lưu 1 vector embedding:  Brad Pitt_embedding.pt
Đã lưu 1 vector embedding:  Hugh Jackman_embedding.pt
Đã lưu 1 vector embedding:  Megan Fox_embedding.pt


In [ ]:
base_recognizer = BaselineRecognizer(
    blazeface_path=blazeface_path,
    anchors_path=anchors_path,
    embeddings_folder=vector_db,
    threshold=0.95
)

evaluate_recognition(test_img, base_recognizer)

[Angelina Jolie_1.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.5687538981437683, Time: 0.126s
[Angelina Jolie_2.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.8584478497505188, Time: 0.114s
[Angelina Jolie_3.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.641179621219635, Time: 0.141s
[Angelina Jolie_4.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.6177566647529602, Time: 0.121s
[Angelina Jolie_5.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.7159461975097656, Time: 0.117s
[Angelina Jolie_6.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.6132485270500183, Time: 0.148s
[Angelina Jolie_7.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.6602413654327393, Time: 0.115s
[Angelina Jolie_8.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.8319584131240845, Time: 0.141s
[Angelina Jolie_9.jpg] → GT: Angelina Jolie, Pred: Angelina Jolie, Dist: 0.7751425504684448, Time: 0.105s
[Angelina Jolie_10.jpg] → GT: Angelina Jolie, P

### DECOGNIZE WITH CAMERA